In [41]:
import numpy as np
import pandas as pd
import math as m
from einops import rearrange
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import tqdm


### Preperation (NOTE: Using meter as unit)

In [69]:
# Utility functions
# convert pointcloud from cartisean coordinate to spherical coordinate
def cart2sph(xyz):
    x = xyz[:,0]
    y = xyz[:,1]
    z = xyz[:,2]
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(list(XsqPlusYsq + z**2))
    elev = np.arctan2(list(z), np.sqrt(list(XsqPlusYsq)))
    pan = np.arctan2(list(x), list(y))

    output = np.array([r, elev, pan])
    return rearrange(output, 'a b -> b a') #take transpose

In [44]:
class LiDAR_NeRF(nn.Module):
    def __init__(self, embedding_dim_pos = 10, embedding_dim_dir = 4, hidden_dim = 256):
        super(LiDAR_NeRF, self).__init__()
        self.embedding_dim_dir = embedding_dim_dir
        self.embedding_dim_pos = embedding_dim_pos
        self.block1 = nn.Sequential(
            nn.Linear(embedding_dim_pos * 6 + 3 + embedding_dim_dir * 4 + 2, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(embedding_dim_pos * 6 + 3 + embedding_dim_dir * 4 + 2 + hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim,1)
        )
        
    @staticmethod
    def positional_encoding(x, L):
        out = [x]
        for j in range(L):
            out.append(torch.sin(2 ** j * x))
            out.append(torch.cos(2 ** j * x))
        return torch.cat(out, dim=1)

    def forward(self, o, d):
        emb_x = self.positional_encoding(o, self.embedding_dim_pos)
        emb_d = self.positional_encoding(d, self.embedding_dim_dir)
        input = rearrange([emb_x,emb_d], 'a b -> (a b)')
        temp = self.block1(input)
        input2 = rearrange([temp, input ], 'a b -> (a b)')
        output = self.block2(input2)
        return output

In [45]:
def lossBCE(rendered_value, actual_value): 
    loss_bce = nn.CrossEntropyLoss()
    loss = loss_bce(rendered_value, actual_value)
    return loss

In [ ]:
def get_sample_positions(origin, angles, ground_truth_distance):
    elev = angles[:,0]
    pan = angles[:,1]
    dir_x = np.cos(elev)*np.cos(pan)
    dir_y = np.cos(elev)*np.sin(pan)
    dir_z = np.sin(elev)
       


    pos = origins + directions * ground_truth_distance * seed   #positions = origin + portion of distance * direction 


    seed = torch.rand([len(batch), 10])
    
    pos = 0 #place holder


    return pos

In [56]:
def train(model, optimizer, scheduler, dataloader, device = 'cpu', epoch = int(1e5)):
    training_losses = []
    for _ in tqdm(range(epoch)):
        for batch in dataloader:
            # parse the batch
            ground_truth_distance = batch[:,0]
            angles = batch[:,1:3]
            origin = batch[:,:3:]
            
            sample_position = get_sample_positions(origin, angles)
            rendered_value = model(sample_position,angles) 

            
            
            sigmoid = nn.sigmoid()
            actual_value = sigmoid(-ground_truth_distance*seed-ground_truth_distance)
            
            loss = lossBCE(rendered_value, actual_value)  # + lossEikonal(model)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            training_losses.append(loss.item())
        scheduler.step()
    return loss
    

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")
points = loadData()
data_matrix = prepareData(points)
training_dataset = torch.from_numpy(data_matrix)
data_loader = DataLoader(training_dataset, batch_size=1024, shuffle = True)
model = LiDAR_NeRF(hidden_dim=256).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2, 4, 8], gamma=0.5)
train(model, optimizer, scheduler, data_loader, epoch = 16, device=device)


Using cuda device


(1012570, 6)